# Output In The Construction Industry

-----

### Requirements

We're looking to transform **tabs 1 and 2 only**.

#### Observations & Dimensions

The `observations` should be apparent.

The required dimensions are:

* **Geography** - it's all UK level data (the code for UK is "K02000001")
* **Time** - either a simple year, or a year followed by a quarter, i.e "2002 Q2"
* **Adjustment** - either seasonal or non-seasonal
* **Business** - concatenation of rows 6-9 around hyphens. i.e 'Other New Work - Excluding Infrastructure - Private Commercial'
* **CDID** - ONS specific 4 letter codes. Typically on row 10 

-----
Notes:

* Getting the **Business** dimension cleanly is going to be tricky (read - cant see an obvious way to do it), I'd perhaps leave this one until last.
* It's always worth getting the file out of /sources and having a look over.
* You can't really take CDID as a dimension (dimension items needs to be repeating, not unqiue), it's a good exercise though as if doing this for real we'd likely be taking it as meta/supporting data.

In [1]:
%cd mock-transformations/

/workspace/mock-transformations


In [2]:
from databaker.framework import *
import pandas as pd
import numpy as np

tabs = loadxlstabs("./sources/OIC.xls") # load tabs

Loading ./sources/OIC.xls which has size 759808 bytes
Table names: ['Cover Sheet', 'Contents', 'Table 1a', 'Table 1b', 'Table 2a', 'Table 2b', 'Table 3a', 'Table 3b', 'Table 3c', 'Table 3d', 'Table 4', 'Table 4a', 'Table 5a', 'Table 5b', 'Table 6a', 'Table 7', 'Table 8', 'Table 9', 'Table 10', 'Table 11']


# Thar be dragons
The hierarchy associated with the Business sector and the CDID should be treated as a seperate transform, and then joined together at the end. This will make the transform simpler.

## Transform steps
1. Process headers as a dataset with the CDIDs as observations
2. Process observations as a dataset with the CDIDs as a dimension
3. Merge headers and observations on the CDIDs to create a new complete dataframe

### Headers/Observations
|       | Column A | Column B      | Column C  | Column D      |  Column E     |   Column F    |
|-------|----------|---------------|-----------|---------------|---------------|---------------|
| Row 1 | YYYY MMM | Category A    |           | Category B    |               | Category C    |
| Row 2 | YYYY MMM | Subcategory 1 |           | Subcategory 2 | Subcategory 3 | Subcategory 4 |
| Row 3 | YYYY MMM | Detail i      | Detail ii | Detail iii    | Detail iv     | Detail v      |
| Row 4 | YYYY MMM | Code 1        | Code 2    | Code 3        | Code 4        | Code 5        |
| Row 5 | YYYY MMM | OBS v         | OBS w     | OBS x         | OBS y         | OBS z         |

* Codes 1 through 5 should be treated as the observations, and the Categories, Subcategories, Details should be treated as a dimension
* The acutal Observations (Row 5 and below) should be lined with the CDID (e.g. Codes 1-5) and the dates.
* Provide up-to three levels of business classification dimentions

## Other notes
* Units should be confirmed that these values are indexed to 2016=100.

In [3]:
def fetch_headers(tab) -> pd.DataFrame:
    # We're futorunate that all the header information is the same
    business_level1 = tab.excel_ref("C6").expand(RIGHT)
    business_level2 = tab.excel_ref("C7").expand(RIGHT)
    business_level3 = tab.excel_ref("C8").expand(RIGHT)

    # These are the CDIDs
    observations = tab.excel_ref("C10").expand(RIGHT).is_not_blank()

    dimensions = [HDim(business_level1, "business_level1", CLOSEST, LEFT),
                  HDim(business_level2, "business_level2", CLOSEST, LEFT),
                  HDim(business_level3, "business_level3", CLOSEST, LEFT)]
    
    # Convert to dataframe
    headers_and_codes = ConversionSegment(tab, dimensions, observations).topandas()
    # Convert empty strings to nans
    headers_and_codes = headers_and_codes.replace(r'', np.nan)
    # Transpose
    headers_only = headers_and_codes.transpose()
    # Omit the codes and fill down on nans
    headers_only = headers_only.iloc[2:].fillna(method='ffill')
    # Transpose again and fill down on nans
    headers_only = headers_only.transpose().fillna(method='ffill')

    # Compare values between business_level2 and busines_level3, clear dupes on latter
    mask = (headers_only.loc[:,'business_level3'] == headers_only.loc[:,'business_level2'])
    headers_only.loc[mask, 'business_level3'] = np.nan

    # Compare values between business_level1 and busines_level2, clear dupes on latter
    mask = (headers_only.loc[:,'business_level2'] == headers_only.loc[:,'business_level1'])
    headers_only.loc[mask, 'business_level2'] = np.nan

    # Join
    headers_and_codes = pd.concat([headers_only, headers_and_codes.iloc[:,1]], axis=1)
    headers_and_codes.rename(columns={"DATAMARKER": "CDID"}, inplace=True)
    return headers_and_codes
    # This is likely broken as the levels aren't closest left. Check the output
    

In [4]:
def fetch_contents(tab) -> pd.DataFrame:
    # Start of the CDIDs is the same throughout all the tabs
    cdids = tab.excel_ref("C10").expand(RIGHT).is_not_blank()

    # There are up-to two levels of the time dimension
    time_level1 = tab.excel_ref("A11").expand(DOWN).is_not_blank()
    time_level2 = tab.excel_ref("B11").expand(DOWN)

    # Put the 🍁 syrup away, no waffle here sadly
    observations = tab.excel_ref("C11").expand(DOWN).expand(RIGHT).is_not_blank()

    dimensions = [HDim(time_level1, "time_level1", CLOSEST, ABOVE),
                  HDim(time_level2, "time_level2", DIRECTLY, LEFT),
                  HDim(cdids, "CDID", DIRECTLY, ABOVE),
                  HDimConst("units", tab.excel_ref("P5").value),
                  HDimConst("adjustment", tab.excel_ref("C2").value),
                  HDimConst("geography", "K02000001")]
    
    return ConversionSegment(tab, dimensions, observations).topandas() #.replace(r'', np.nan)

In [5]:
# All the hard work is now done
work = list()
for tab in tabs:
    if tab.name not in ['Table 1a', 'Table 1b', 'Table 2a', 'Table 2b']:
        continue
    
    df = fetch_contents(tab).set_index("CDID").join(fetch_headers(tab).set_index("CDID"), how="left", on="CDID")

    work.append(df)


In [6]:
# Join into a single dataframe
output = pd.concat(work)
# Drop duplicated column names
# output = output.loc[:,~output.columns.duplicated()]
# We played with the indexes to clear things up, so release it to the dimensions
output.reset_index(inplace=True)

In [7]:
output.head()

,CDID,OBS,time_level1,time_level2,units,adjustment,geography,business_level1,business_level2,business_level3
0,MV36,38.9,1997.0,,2016=100,CONSTRUCTION OUTPUT IN GREAT BRITAIN: VOLUME S...,K02000001,New Housing,Public,NaN
1,MV37,51.9,1997.0,,2016=100,CONSTRUCTION OUTPUT IN GREAT BRITAIN: VOLUME S...,K02000001,New Housing,Private,NaN
2,MVL7,49.9,1997.0,,2016=100,CONSTRUCTION OUTPUT IN GREAT BRITAIN: VOLUME S...,K02000001,Total Housing,NaN,NaN
3,MV38,69.6,1997.0,,2016=100,CONSTRUCTION OUTPUT IN GREAT BRITAIN: VOLUME S...,K02000001,Other New Work,Infrastruc-ture,NaN
4,MV39,50.0,1997.0,,2016=100,CONSTRUCTION OUTPUT IN GREAT BRITAIN: VOLUME S...,K02000001,Other New Work,Excluding Infrastructure,Public


In [8]:
# Adjustments
# Seasonal adjustments are not the only adjustment to this data, but I digress
searepl = {"NON-SEASONALLY": "Non-seasonal",
           "SEASONALLY": "Seasonal"}
for x,y in searepl.items():
    output.loc[output["adjustment"].str.contains(x),"adjustment"] = y


In [9]:
output

,CDID,OBS,time_level1,time_level2,units,adjustment,geography,business_level1,business_level2,business_level3
0,MV36,38.9,1997.0,,2016=100,Seasonal,K02000001,New Housing,Public,NaN
1,MV37,51.9,1997.0,,2016=100,Seasonal,K02000001,New Housing,Private,NaN
2,MVL7,49.9,1997.0,,2016=100,Seasonal,K02000001,Total Housing,NaN,NaN
3,MV38,69.6,1997.0,,2016=100,Seasonal,K02000001,Other New Work,Infrastruc-ture,NaN
4,MV39,50.0,1997.0,,2016=100,Seasonal,K02000001,Other New Work,Excluding Infrastructure,Public
...,...,...,...,...,...,...,...,...,...,...
12707,MV4J,1715.0,2019.0,July,(£million),Non-seasonal,K02000001,Repair and Maintenance,Housing,Private
12708,MV4K,2346.0,2019.0,July,(£million),Non-seasonal,K02000001,Repair and Maintenance,Housing,Total
12709,MV4L,2512.0,2019.0,July,(£million),Non-seasonal,K02000001,Repair and Maintenance,Non Housing R&M,NaN
12710,MV4M,4858.0,2019.0,July,(£million),Non-seasonal,K02000001,All Repair and Maintenance,NaN,NaN


In [10]:
# Fix business_level2 "Infrastuc-ture" > "Infrastructure"
output.loc[output["business_level2"] == "Infrastruc-ture", "business_level2"] = "Infrastructure"

In [12]:
# time column fixes (blanks, ints to strs, and concatenation)
output["time_level1"] = output["time_level1"].astype(float).astype(int).astype(str)
output["time_level2"] = output["time_level2"].astype(str)
output["time"] = output[["time_level1","time_level2"]].apply(lambda x: ' '.join(x), axis=1)
output.drop(labels=["time_level1", "time_level2"], axis=1, inplace=True)

In [36]:
# business levels
# Level 1 + Level 2, remove NaNs
output["business"] = output["business_level1"].str.cat(output["business_level2"], sep=" - ", na_rep="").str.replace(' - $', '')
# + Level 3, remove NaNs
output["business"] = output["business"].str.cat(output["business_level3"], sep=" - ", na_rep="").str.replace(' - $', '')

/usr/local/lib/python3.7/site-packages/numpy/core/fromnumeric.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [37]:
output

,CDID,OBS,units,adjustment,geography,business_level1,business_level2,business_level3,time,business
0,MV36,38.9,2016=100,Seasonal,K02000001,New Housing,Public,NaN,1997,New Housing - Public
1,MV37,51.9,2016=100,Seasonal,K02000001,New Housing,Private,NaN,1997,New Housing - Private
2,MVL7,49.9,2016=100,Seasonal,K02000001,Total Housing,NaN,NaN,1997,Total Housing
3,MV38,69.6,2016=100,Seasonal,K02000001,Other New Work,Infrastructure,NaN,1997,Other New Work - Infrastructure
4,MV39,50.0,2016=100,Seasonal,K02000001,Other New Work,Excluding Infrastructure,Public,1997,Other New Work - Excluding Infrastructure - Pu...
...,...,...,...,...,...,...,...,...,...,...
12707,MV4J,1715.0,(£million),Non-seasonal,K02000001,Repair and Maintenance,Housing,Private,2019 July,Repair and Maintenance - Housing - Private
12708,MV4K,2346.0,(£million),Non-seasonal,K02000001,Repair and Maintenance,Housing,Total,2019 July,Repair and Maintenance - Housing - Total
12709,MV4L,2512.0,(£million),Non-seasonal,K02000001,Repair and Maintenance,Non Housing R&M,NaN,2019 July,Repair and Maintenance - Non Housing R&M
12710,MV4M,4858.0,(£million),Non-seasonal,K02000001,All Repair and Maintenance,NaN,NaN,2019 July,All Repair and Maintenance
